In [ ]:
# mount drive for convenience (stored images here in a folder)
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [ ]:
# !pip install bert-for-tf2

In [ ]:
# import all necessary libraries
import os
os.environ["NLTK_DATA"] = "/content/drive/MyDrive/downloads/nltk"

import sys
sys.path.append("/content/drive/MyDrive/python3.7_packages/")

import re
import string
import warnings
import numpy as np
import pandas as pd
import bert
import nltk
import os
import warnings
import argparse
import bert
import requests
from PIL import Image
from io import BytesIO
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelBinarizer
import tensorflow_hub as hub
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import VGG16
stop = stopwords.words("english")
warnings.filterwarnings("ignore")


stop = stopwords.words("english")
warnings.filterwarnings("ignore")
pd.set_option("display.max_colwidth", 200)
pd.set_option('display.max_columns', None)

In [ ]:
# load data into pandas dataframes
# comments = pd.read_csv("/content/drive/MyDrive/Fakeddit/all_comments.tsv/all_comments.tsv", sep="\t")
multimodal_train = pd.read_csv("/content/drive/MyDrive/Fakeddit/multimodal_only_samples-20210510T062706Z-001/multimodal_only_samples/multimodal_train.tsv", sep="\t")

In [ ]:
comments.head(2)

,Unnamed: 0,id,author,body,isTopLevel,parent_id,submission_id,ups
0,0,f4deplg,Haus42,"Scroll, scroll, scroll. Pause. Scroll back up. Check which subreddit this article is in. Nod in approval. Upvote. Continue scrolling.",True,t3_dkdml1,dkdml1,162.0
1,1,f4d79bi,PotatoPrince84,"A lot of the people who felt quite strongly about the fake news being spread on Facebook have been pushing for this for a while now, but a lot of critics have raised the valid point that mere mort...",True,t3_dkdml1,dkdml1,84.0


In [ ]:
comments.shape

(10697533, 8)

In [ ]:
multimodal_train.shape

(564000, 16)

In [ ]:
multimodal_train.head(5)

,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
0,Alexithymia,my walgreens offbrand mucinex was engraved with the letters mucinex but in a different order,1.551641e+09,i.imgur.com,True,awxhir,https://external-preview.redd.it/WylDbZrnbvZdBpgfa3ntxYf17CBHndiJWHylVm2j_nY.jpg?width=320&crop=smart&auto=webp&s=449659a10792de4d55c2f27d2176fdc8bc66e72a,NaN,2.0,12,mildlyinteresting,My Walgreens offbrand Mucinex was engraved with the letters Mucinex but in a different order,0.84,1,0,0
1,VIDCAs17,this concerned sink with a tiny hat,1.534727e+09,i.redd.it,True,98pbid,https://preview.redd.it/wsfx0gp0f5h11.jpg?width=320&crop=smart&auto=webp&s=6267a3ae11e616453e129c58ae34635b98c4ab37,NaN,2.0,119,pareidolia,This concerned sink with a tiny hat,0.99,0,2,2
2,prometheus1123,hackers leak emails from uae ambassador to us,1.496511e+09,aljazeera.com,True,6f2cy5,https://external-preview.redd.it/6fNhdbc6K1vFAZX95ed-ZMLDlASqmpixvAxmXqHhfHs.jpg?width=320&crop=smart&auto=webp&s=f263f843e91b7dbea1d1157031f7ea0d555186ed,NaN,1.0,44,neutralnews,Hackers leak emails from UAE ambassador to US,0.92,1,0,0
3,NaN,puppy taking in the view,1.471341e+09,i.imgur.com,True,4xypkv,https://external-preview.redd.it/HLtVNhTR6wtYtnvL8RKfmZFdKQ7tVz42WCiH3jSGiIk.jpg?width=320&crop=smart&auto=webp&s=2d60c452b15d7d86b46b65e06da9a6e42f10d5df,NaN,26.0,250,photoshopbattles,PsBattle: Puppy taking in the view,0.95,1,0,0
4,3rikR3ith,i found a face in my sheet music too,1.525318e+09,i.redd.it,True,8gnet9,https://preview.redd.it/ri7ut2wn8kv01.jpg?width=320&crop=smart&auto=webp&s=d44a419480c2073a2f0f3fa5f3701b308271331d,NaN,2.0,13,pareidolia,I found a face in my sheet music too!,0.84,0,2,2


In [ ]:
multimodal_train.columns

Index(['author', 'clean_title', 'created_utc', 'domain', 'hasImage', 'id',
       'image_url', 'linked_submission_id', 'num_comments', 'score',
       'subreddit', 'title', 'upvote_ratio', '2_way_label', '3_way_label',
       '6_way_label'],
      dtype='object')

In [ ]:
multimodal_train["6_way_label"].value_counts()

0    222081
4    167857
2    107221
1     33481
5     21576
3     11784
Name: 6_way_label, dtype: int64

In [ ]:
# df = pd.merge(multimodal_train, comments, on="id", how="left")
df = multimodal_train

In [ ]:
df.shape

(564000, 16)

In [ ]:
# filter df which has image urls
df = df[~df["image_url"].isna()]
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
0,Alexithymia,my walgreens offbrand mucinex was engraved with the letters mucinex but in a different order,1.551641e+09,i.imgur.com,True,awxhir,https://external-preview.redd.it/WylDbZrnbvZdBpgfa3ntxYf17CBHndiJWHylVm2j_nY.jpg?width=320&crop=smart&auto=webp&s=449659a10792de4d55c2f27d2176fdc8bc66e72a,NaN,2.0,12,mildlyinteresting,My Walgreens offbrand Mucinex was engraved with the letters Mucinex but in a different order,0.84,1,0,0
1,VIDCAs17,this concerned sink with a tiny hat,1.534727e+09,i.redd.it,True,98pbid,https://preview.redd.it/wsfx0gp0f5h11.jpg?width=320&crop=smart&auto=webp&s=6267a3ae11e616453e129c58ae34635b98c4ab37,NaN,2.0,119,pareidolia,This concerned sink with a tiny hat,0.99,0,2,2
2,prometheus1123,hackers leak emails from uae ambassador to us,1.496511e+09,aljazeera.com,True,6f2cy5,https://external-preview.redd.it/6fNhdbc6K1vFAZX95ed-ZMLDlASqmpixvAxmXqHhfHs.jpg?width=320&crop=smart&auto=webp&s=f263f843e91b7dbea1d1157031f7ea0d555186ed,NaN,1.0,44,neutralnews,Hackers leak emails from UAE ambassador to US,0.92,1,0,0
3,NaN,puppy taking in the view,1.471341e+09,i.imgur.com,True,4xypkv,https://external-preview.redd.it/HLtVNhTR6wtYtnvL8RKfmZFdKQ7tVz42WCiH3jSGiIk.jpg?width=320&crop=smart&auto=webp&s=2d60c452b15d7d86b46b65e06da9a6e42f10d5df,NaN,26.0,250,photoshopbattles,PsBattle: Puppy taking in the view,0.95,1,0,0
4,3rikR3ith,i found a face in my sheet music too,1.525318e+09,i.redd.it,True,8gnet9,https://preview.redd.it/ri7ut2wn8kv01.jpg?width=320&crop=smart&auto=webp&s=d44a419480c2073a2f0f3fa5f3701b308271331d,NaN,2.0,13,pareidolia,I found a face in my sheet music too!,0.84,0,2,2


In [ ]:
# df[~df["body"].isna()]

In [ ]:
# save images to the desired folder
def save_images_to_folder(image_urls, names, folder_path, extension):
    
    not_available = []
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    for i in range(len(image_urls)):
        
        file_name = "{}.{}".format(names[i], extension)
        file_name = os.path.join(folder_path, file_name)

        try:
          image = requests.get(image_urls[i]).content
          image = Image.open(BytesIO(image)).convert("RGB")
          image.save(file_name)

        except Exception as e:
          not_available.append(i)
    
    return not_available

In [ ]:
image_folder = "/content/drive/MyDrive/Fakeddit/multimodal_only_samples-20210510T062706Z-001/images"
# not_found = save_images_to_folder(df["image_url"].tolist()[:1000], df["id"].tolist()[:1000], image_folder, "jpg")

In [ ]:
# load the image and preprocess it

images = []
indices = []
for i in os.listdir(image_folder)[:1000]:
  image_path = os.path.join(folder, i)
  image = load_img(image_path, target_size=(224, 224, 3))
  image = img_to_array(image)

  images.append(image)
  indices.append(i.split("_")[1].split(".")[0])

indices = list(map(int, indices))
images = np.array(images)
images.shape

In [ ]:
# create train df
train_df = df.iloc[indices]
train_df.reset_index(inplace=True, drop=True)

In [ ]:
# num labels for multi class classification
num_labels = len(train_df["6_way_label"].value_counts())
num_labels

In [ ]:
one_hot = LabelBinarizer().fit(train_df["6_way_label"].unique())
labels = one_hot.fit_transform(train_df["6_way_label"])

### BERT EMBEDDINGS

In [ ]:
# function to encode texts into bert embeddings
def bert_encode(tokenizer, bert_layer, texts, max_len, mode="pooled"):

    def convert_tokens(tokenizer, text, max_len):

        tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

        if len(tokens) <= max_len - 2:
            tokens.append(tokenizer.convert_tokens_to_ids(["[SEP]"])[0])
        else:
            tokens = tokens[:max_len - 2] + tokenizer.convert_tokens_to_ids(["[SEP]"])

        return tokens
    
    sentence = tf.ragged.constant([convert_tokens(tokenizer, i, max_len) for i in texts])
    cls_token = [tokenizer.convert_tokens_to_ids(["[CLS]"])] * len(texts)
    input_word = tf.concat([cls_token, sentence], axis=-1)

    mask = tf.ones_like(input_word).to_tensor()
    type_id = tf.zeros_like(input_word).to_tensor()

    inputs = {
    'input_word_ids': input_word.to_tensor(),
    'input_mask': mask,
    'input_type_ids': type_id}

    bert_embeddings = bert_layer(inputs)

    if mode == "pooled":
        return bert_embeddings["pooled_output"]

    else:
        return bert_embeddings["sequence_output"]

In [ ]:
# load the small bert model and configure it
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2", trainable=False)
bert_tokenizer = bert.bert_tokenization.FullTokenizer
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenizer(vocabulary_file, to_lower_case)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'.
INFO:absl:Downloaded https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2, Total size: 19.27MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'.


In [ ]:
# embeddings created for the clean title column in train_df
bert_embeddings = bert_encode(tokenizer, bert_layer, train_df["clean_title"].tolist(), max_len=300, mode="sequence")
bert_embeddings.shape

TensorShape([1000, 52, 128])

In [ ]:
# function to build deep learning model
def classification_model(max_len, num_labels):
    
    # text model
    text_input = Input(shape=(max_len, 128, ))
    layer = Bidirectional(LSTM(64, return_sequences=True))(text_input)
    layer = Bidirectional(LSTM(32))(layer)
    text_flat = Flatten()(layer)

    # image model
    vgg_16 = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    vgg_16.trainable = False
    vgg_output = vgg_16.output
    image_flat = Flatten()(vgg_output)
    image_output = Dense(64, activation="relu")(image_flat)

    # concatenate the inputs to form a single flattened vector
    final_input = Concatenate(axis=1)([text_flat, image_output])
    dense = Dense(32, activation="relu")(final_input)
    output = Dense(num_labels, activation="softmax")(dense)
    
    # return final model
    model = Model(inputs=[text_input, vgg_16.input], outputs=output)
    return model

In [ ]:
# create and compile model
model = classification_model(bert_embeddings.shape[1], num_labels)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics="accuracy")

text output: KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='flatten/Reshape:0', description="created by layer 'flatten'")
58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
______________________________________________________________________________________________

In [ ]:
# train the model on the dataset
train = model.fit(x=[bert_embeddings, images], y=labels, epochs=10, verbose=True, batch_size=128, validation_split=0.2)

Epoch 1/10
7/7 [==============================] - 4s 595ms/step - loss: 2.3814 - accuracy: 0.6375 - val_loss: 4.7439 - val_accuracy: 0.3650
Epoch 2/10
7/7 [==============================] - 4s 599ms/step - loss: 1.1158 - accuracy: 0.7550 - val_loss: 4.8541 - val_accuracy: 0.4800
Epoch 3/10
7/7 [==============================] - 4s 603ms/step - loss: 0.4213 - accuracy: 0.8988 - val_loss: 4.2275 - val_accuracy: 0.4200
Epoch 4/10
7/7 [==============================] - 4s 610ms/step - loss: 0.2279 - accuracy: 0.9287 - val_loss: 4.5364 - val_accuracy: 0.4750
Epoch 5/10
7/7 [==============================] - 4s 604ms/step - loss: 0.1661 - accuracy: 0.9688 - val_loss: 5.1967 - val_accuracy: 0.4800
Epoch 6/10
7/7 [==============================] - 4s 612ms/step - loss: 0.0890 - accuracy: 0.9887 - val_loss: 4.8181 - val_accuracy: 0.4800
Epoch 7/10
7/7 [==============================] - 4s 619ms/step - loss: 0.0340 - accuracy: 0.9925 - val_loss: 4.7103 - val_accuracy: 0.4550
Epoch 8/10
7/7 [====

## TENSORFLOW DATA PIPILINE

In [ ]:
# text file and images locations
text_path = "/content/drive/MyDrive/Fakeddit/multimodal_only_samples-20210510T062706Z-001/multimodal_only_samples/multimodal_train.tsv"
image_folder = "/content/drive/MyDrive/Fakeddit/multimodal_only_samples-20210510T062706Z-001/images"

In [ ]:
# initialize hyperparameters
batch_size = 128
epochs = 10
num_labels = 6
sample_count = 500

In [ ]:
# read the text data into Tensorflow Data API 
data = tf.data.experimental.make_csv_dataset(text_path, batch_size=1, select_columns=["clean_title", "id", "image_url", "num_comments", "6_way_label"], field_delim="\t", shuffle=False)

In [ ]:
data = data.take(sample_count).filter(lambda x: tf.math.reduce_all(tf.math.not_equal(x["image_url"], "")))

In [ ]:
# function to parse images
def parse_image(filename):
  
  img = tf.io.read_file(filename)
  img = tf.image.decode_jpeg(img, channels=3)
  img =  tf.image.resize(img, [224, 224])

  return img

In [ ]:
# get images and their directories
images_path = data.map(lambda x: x["id"])
images_path = images_path.map(lambda x: tf.strings.join([image_folder, "/", x, ".jpg"])[0])
images = images_path.map(parse_image)

In [ ]:
labels = data.map(lambda x: x["6_way_label"])
labels = labels.map(lambda x: tf.one_hot(x, num_labels))
labels = labels.map(lambda x: tf.squeeze(x, axis=0))

In [ ]:
# sanity check for one hot encode
for i in labels.take(1):
  print(i)

tf.Tensor([1. 0. 0. 0. 0. 0.], shape=(6,), dtype=float32)


In [ ]:
# function to encode texts into bert embeddings
def bert_encode_modified(tokenizer, data, bert_layer, max_len, mode="sequence"):

    def convert_tokens(tokenizer, text, max_len):

        tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

        if len(tokens) <= max_len - 2:
            tokens.append(tokenizer.convert_tokens_to_ids(["[SEP]"])[0])
        else:
            tokens = tokens[:max_len - 2] + tokenizer.convert_tokens_to_ids(["[SEP]"])

        return tokens
    sentence = tf.ragged.constant([convert_tokens(tokenizer, s["clean_title"].numpy()[0], max_len) for s in data])
    cls_token = [tokenizer.convert_tokens_to_ids(["[CLS]"])] * sentence.shape[0]
    input_word = tf.concat([cls_token, sentence], axis=-1)

    mask = tf.ones_like(input_word).to_tensor()
    type_id = tf.zeros_like(input_word).to_tensor()

    inputs = {
    'input_word_ids': input_word.to_tensor(),
    'input_mask': mask,
    'input_type_ids': type_id}

    bert_embeddings = bert_layer(inputs)

    if mode == "pooled":
        return bert_embeddings["pooled_output"]

    else:
        return bert_embeddings["sequence_output"]

In [ ]:
# text embeddings from BERT and convert to TF Dataset format
bert_embeddings = bert_encode_modified(tokenizer, data, bert_layer, max_len=300, mode="sequence")
text_embeddings = tf.data.Dataset.from_tensor_slices(bert_embeddings)

In [ ]:
# zip the inputs and labels into a single dataset
dataset = tf.data.Dataset.zip(((text_embeddings, images), labels)).shuffle(100, reshuffle_each_iteration=False)

In [ ]:
# split data into train and validation split
val_split = int(bert_embeddings.shape[0] * 0.2)
val_ds = dataset.take(val_split).batch(32)
train_ds = dataset.skip(val_split).batch(32)

In [ ]:
val_split

99

In [ ]:
# create a new model or load an existing model and compile it
model_folder = os.path.join("/content/drive/MyDrive/Fakeddit", "models")

if not os.path.exists(model_folder):
  os.makedirs(model_folder)

model_file = os.path.join(model_folder, "model.h5")

if os.path.exists(model_file):
  model = load_model

else:
  model = classification_model(bert_embeddings.shape[1], num_labels)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [ ]:
# train the model
train = model.fit(train_ds, epochs=epochs, verbose=True, validation_data=val_ds)

Epoch 1/10
13/13 [==============================] - 15s 765ms/step - loss: 15.3908 - accuracy: 0.3431 - val_loss: 6.1282 - val_accuracy: 0.4141
Epoch 2/10
13/13 [==============================] - 5s 302ms/step - loss: 7.0169 - accuracy: 0.5115 - val_loss: 5.2482 - val_accuracy: 0.4242
Epoch 3/10
13/13 [==============================] - 5s 305ms/step - loss: 0.7989 - accuracy: 0.8924 - val_loss: 5.0802 - val_accuracy: 0.4343
Epoch 4/10
13/13 [==============================] - 5s 307ms/step - loss: 0.2544 - accuracy: 0.9360 - val_loss: 5.4197 - val_accuracy: 0.4444
Epoch 5/10
13/13 [==============================] - 5s 306ms/step - loss: 0.0383 - accuracy: 0.9853 - val_loss: 5.2019 - val_accuracy: 0.4747
Epoch 6/10
13/13 [==============================] - 5s 306ms/step - loss: 0.0168 - accuracy: 0.9973 - val_loss: 5.2720 - val_accuracy: 0.4646
Epoch 7/10
13/13 [==============================] - 5s 306ms/step - loss: 0.0090 - accuracy: 0.9995 - val_loss: 5.2966 - val_accuracy: 0.4545
Epoc

In [ ]:
# save the model to directory
model.save(model_file)